In [1]:
#Importing needed packages
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import pandas as pd
import B_funktioner
import pydst as pydst
dst = pydst.Dst(lang='en')



In [11]:
#Collection of datasets and cleaning
#This first section collects datasets from DST and cleans in order to get the wanted variables
#Tænker at dette skal gøres mere udførligt osv. Det er bare lige for vores eget overbliks skyld
#Måske skal vi overveje at have helt i starten noget der undersøger hvilke datasæt vi vil hente ala nedenstående

dst.get_subjects()

dst.get_tables(subjects=['02']).head(10)
dst.get_variables(table_id='FOLK1A')

dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id='RAS200')




,id,text,elimination,time,map,values
0,OMRÅDE,region,True,False,denmark_municipality_07,"[{'id': '000', 'text': 'All Denmark'}, {'id': ..."
1,HERKOMST,ancestry,True,False,NaN,"[{'id': '00', 'text': 'Total'}, {'id': '10', '..."
2,ALDER,age,True,False,NaN,"[{'id': 'TOT', 'text': 'Age, total'}, {'id': '..."
3,KØN,sex,True,False,NaN,"[{'id': 'TOT', 'text': 'Total'}, {'id': 'M', '..."
4,BEREGNING,frequency,False,False,NaN,"[{'id': 'BFK', 'text': 'Employment rate'}, {'i..."
5,Tid,time,False,True,NaN,"[{'id': '2008', 'text': '2008'}, {'id': '2009'..."


In [2]:
#downloading the population of the danish municipalities
dst.get_subjects()
dst.get_tables(subjects=['02']).head(10)
dst.get_variables(table_id='FOLK1A')

variables = {'OMRÅDE':['*'],'Tid':['*']}
pop = dst.get_data(table_id = 'FOLK1A', variables=variables)
pop.head(10)

pop = B_funktioner.initial_rename(pop,'population')
pop = B_funktioner.only_keep_municipalities_and_years(pop,0,2008,2018)
pop = B_funktioner.sort_reset(pop)

pop


,municipality,year,KØN,ALDER,CIVILSTAND,population
0,Aabenraa,2008,Total,Total,Total,60189
1,Aabenraa,2009,Total,Total,Total,60392
2,Aabenraa,2010,Total,Total,Total,59978
3,Aabenraa,2011,Total,Total,Total,59795
4,Aabenraa,2012,Total,Total,Total,59600
...,...,...,...,...,...,...
1084,Ærø,2014,Total,Total,Total,6393
1085,Ærø,2015,Total,Total,Total,6276
1086,Ærø,2016,Total,Total,Total,6290
1087,Ærø,2017,Total,Total,Total,6177


In [10]:
#downloading the employment data of the danish municipalities

variables={'OMRÅDE':['*'], 'BEREGNING':['*'], 'TID':['*']}
empl = dst.get_data(table_id = 'RAS200', variables=variables)

empl = B_funktioner.initial_rename(empl,'employment rate')
empl = B_funktioner.only_keep_municipalities_and_years(empl,0,2008,2018)
empl = B_funktioner.row_chooser(empl,BEREGNING,'Employment rate')
"""
for val in ['Economic activity rate']: 
    I = empl.BEREGNING.str.contains(val)
    empl = empl.loc[I == False] # keep everything else
"""
empl= B_funktioner.sort_reset(empl)

empl


NameError: name 'BEREGNING' is not defined

,municipality,BEREGNING,year,HERKOMST,ALDER,KØN,population
1,Aabenraa,Employment rate,2008,Total,"Age, total",Total,74.6
3,Aabenraa,Employment rate,2009,Total,"Age, total",Total,71.6
5,Aabenraa,Employment rate,2010,Total,"Age, total",Total,70.9
7,Aabenraa,Employment rate,2011,Total,"Age, total",Total,70.2
9,Aabenraa,Employment rate,2012,Total,"Age, total",Total,69.5
...,...,...,...,...,...,...,...
2168,Ærø,Employment rate,2014,Total,"Age, total",Total,65.1
2171,Ærø,Employment rate,2015,Total,"Age, total",Total,65.8
2172,Ærø,Employment rate,2016,Total,"Age, total",Total,67.0
2175,Ærø,Employment rate,2017,Total,"Age, total",Total,68.1
